No Longer Using!!!!!!

Before I can create a unique word frequency list or translate individual words, I need to know the type of each word in each sentence. For this I can use part-of-speech tagging with an HMM or transformer that has been pre-trained on a french corpus. Two options for french are the Stanford POS pre-trained model which uses their Maximum Entropy model, or a Long-Short-Term Memory Conditional Random Field (LSTM-CRF) pre-trained by huggingface that uses flair contextual string embeddings. Using this method I will add a new column to my csv database that includes all words in the sentence and their type in JSON format.

the camemBERT Flair POS tag list has 69 tags. Once I have each word tagged, I can use the microsoft azure translation API which returns multiple translations, their likelihood and the POS tag of the translation, then I can just map the camemBERT tags to the azure tags and get the most appropriate translation instead of showing multiple translations. Thinking about it this will only work if each translation has a different POS tag, for example une plume is a noun in both senses of the word, so POS tagging won't be an effective way to discern between them, so this probably isn't the best approach. I'll just show multiple translations.

POS French camemBERT Flair tag list: https://huggingface.co/qanastek/pos-french-camembert-flair

In [1]:
import flair
import pandas as pd

c:\Users\Toby Usher\Documents\dev\quivo-app\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
language_code = 'fr'

df = pd.read_csv(f'./output_files/{language_code}/step4_sentences.csv')

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# Load the LSTM-CRF model pre-trained for french from huggingface
model = SequenceTagger.load("qanastek/pos-french")

c:\Users\Toby Usher\Documents\dev\quivo-app\venv\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Toby Usher\.flair\models\pos-french. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


2023-11-04 14:03:59,617 SequenceTagger predicts: Dictionary with 69 tags: <unk>, O, DET, NFP, ADJFP, AUX, VPPMS, ADV, PREP, PDEMMS, NMS, COSUB, PINDMS, PPOBJMS, VERB, DETFS, NFS, YPFOR, VPPFS, PUNCT, DETMS, PROPN, ADJMS, PPER3FS, ADJFS, COCO, NMP, PREL, PPER1S, ADJMP, VPPMP, DINTMS, PPER3MS, PPER3MP, PREF, ADJ, DINTFS, CHIF, XFAMIL, PRELFS, SYM, NOUN, MOTINC, PINDFS, PPOBJMP, NUM, PREFP, PDEMFS, VPPFP, PPER3FP
Sentence[6]: "George Washington est allé à Washington" → ["George"/PROPN, "Washington"/XFAMIL, "est"/AUX, "allé"/VPPMS, "à"/PREP, "Washington"/PROPN]


In [3]:
# Create a sentence
sentence = Sentence("C'est la premiere fois que j'utilise cette methode et il y a des fautes de grammaire et un moirfas qui n'existe pas!")

# Predict tags
model.predict(sentence)

# Print predicted pos tags
print(sentence.to_tagged_string())

Sentence[23]: "C'est la premiere fois que j'utilise cette methode et il y a des fautes de grammaire et un moirfas qui n'existe pas!" → ["C'est"/PREP, "la"/DETFS, "premiere"/ADJFS, "fois"/NFS, "que"/COSUB, "j'utilise"/PREP, "cette"/PDEMFS, "methode"/NFS, "et"/COCO, "il"/PPER3MS, "y"/PPOBJMS, "a"/VERB, "des"/DET, "fautes"/NFP, "de"/PREP, "grammaire"/NFS, "et"/COCO, "un"/DINTMS, "moirfas"/NMS, "qui"/PREL, "n'existe"/VERB, "pas"/ADV, "!"/PUNCT]


This is pretty accurate (Overall accuracy around 98%, but far less so for certain word types), but I need to make sure the labels aren't too specific, otherwise they could split the counts for the same word with similar but not identical meaning across different contexts into separate categories.